In [1]:
from library_buatan.holdout_workflow_comvis import Holdout_Workflow_CV
from library_buatan.holdout_workflow_comvisV2 import Holdout_Workflow_CV2
# from mobilenetv2cbam.mobilenetv2cbam_hilmi import MobileNetV2_CBAM_hilmi
# from mobilenetv2.mobilenetv2 import MobileNetV2
# from basic_workflow_comvis import Pov2ImageDataset_V4

import torch
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torchvision import models
from torchvision.models.mobilenetv3 import _mobilenet_v3_conf

import numpy as np

import pandas as pd

from PIL import Image

import matplotlib.pyplot as plt

In [2]:
import time
import joblib
import optuna
# from basic_workflow_comvis import DfToDataset

/home/andre/.conda/envs/smt11/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# target_names = {
#   "AF"    :0,
#   "AN"    :1,
#   "DI"    :2,
#   "HA"    :3,
#   "NE"    :4,
#   "SA"    :5,
#   "SU"    :6
# }
# target_names = {
#   "c0":0,
#   "c1":1,
#   "c2":2,
#   "c3":3,
#   "c4":4,
#   "c5":5,
#   "c6":6,
#   "c7":7,
#   "c8":8,
#   "c9":9
# }
# target_names = ['Apple___Apple_scab','Apple___Black_rot','Apple___Cedar_apple_rust','Apple___healthy']

target_names = {0:'Apple_scab',
                1:'Black_rot',
                2:'Cedar_apple_rust',
                3:'healthy'}

# print(device)

# workflow = Holdout_Workflow_CV(target_names, device)
workflow = Holdout_Workflow_CV2(target_names, device)
inverted_residual_setting, last_channel = _mobilenet_v3_conf('mobilenet_v3_large')

Using device: cuda:0


In [4]:

mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.5, 0.5, 0.5])

transform_train = transforms.Compose([
    transforms.CenterCrop(size=((224, 224))),
    # transforms.Resize(size=(224, 224)),
    # transforms.RandomAffine(degrees=(-15, 15), shear=(-15, 15)),

#   transforms.Lambda(lambda img: imgaug_transform.augment_image(np.array(img))),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean,std),
])

transform_test = transforms.Compose([
    transforms.CenterCrop(size=((224, 224))),
    # transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean,std),
])

train_dataset = datasets.ImageFolder(root='apple_leaf_dataset/backbone/train_backbone',
                                           transform=transforms.ToTensor()) #transform_train,)
                                        #    target_transform=transforms.Lambda(lambda label: torch.tensor(target_names[label])) )
valid_dataset = datasets.ImageFolder(root='apple_leaf_dataset/backbone/valid_backbone',
                                           transform=transforms.ToTensor())#transform_test,)
                                        #    target_transform=transforms.Lambda(lambda label: torch.tensor(target_names[label])) )
test_dataset = datasets.ImageFolder(root='apple_leaf_dataset/test',
                                           transform=transforms.ToTensor())#transform_test,)


### Uji Coba

In [5]:
# name_file = f"kmu_fed_kfold{n_splits}.{fi}.{epochs}_mobilenetv2_v0.3"
name_model = "mobilenetv3large"
model = models.MobileNetV3(inverted_residual_setting=inverted_residual_setting,
                    last_channel=last_channel,
                    num_classes=4) #MobileNetV2_CBAM_hilmi(10)
str_to_num=False
save_path = f"hasil/{name_model}"

# path_dir = "../resources"
name_data = "apple_leaf_disease"
# path = path_dir+"/isdds_skeleton/90c_v2"
# dataset_list = [path_dir+"/isdds_irhv2_nonpersonsplitv2_holdout/latih_isdds90c_irhv2.csv",
#                 path_dir+"/isdds_irhv2_nonpersonsplitv2_holdout/valid_isdds90c_irhv2.csv",
#                 path_dir+"/isdds_irhv2_nonpersonsplitv2_holdout/uji_isdds90c_irhv2.csv",]
dataset_list2 = [
    train_dataset,
    valid_dataset,
    test_dataset
]

criterion = nn.CrossEntropyLoss()

optim_params = {}
optimizer = optim.Adam
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

patience = 15#30
# counter = 0

# n_splits=5
epochs=1
batch_size= 16#8#64#32

versi = "Adam_default_outchan1280_16batch" #X, coba lr0.001

In [6]:
# workflow.run_workflow(
#     versi=versi, model=model, str_to_num=str_to_num,
#     # init_model=init_mobilenetv2(),
#     # dataset_const=dataset,
#     batch_size=batch_size,
#     # n_splits=n_splits,
#     epochs=epochs,
#     patience=patience,
#     # class_names=class_names,
#     path='',
#     name_model=name_model,
#     name_data=name_data,
#     save_path0=save_path,
#     criterion_const=criterion,
#     optim_const=optimizer, optim_params=optim_params,
#     transform_train=transform_train, transform_test=transform_test,
#     dataset_list=None, #dataset_list
#     dataset_list2=dataset_list2
# )

workflow.run_demo(
    versi=versi, model1=model, str_to_num=str_to_num,
    # init_model=init_mobilenetv2(),
    # dataset_const=dataset,
    batch_size=batch_size,
    # n_splits=n_splits,
    epochs=epochs,
    patience=patience,
    # class_names=class_names,
    path='',
    name_model=name_model,
    name_data=name_data,
    save_path0=save_path,
    criterion_const=criterion,
    optim_const=optimizer, optim_params=optim_params,
    transform_train=transform_train, transform_test=transform_test,
    dataset_list=None, #dataset_list
    dataset_list2=dataset_list2,
    pilihan='y',
)


	Epochs 1
		Best -> epoch: 1
		Training Loss: 0.474
			Training Accuracy: 95.643%
			Training Precision: 95.706%
			Training F1: 95.643%
			Training Recall: 95.643%
		Validation loss: 15.890
			Validation Accuracy: 25.989%
			Validation Precision: 6.754%
			Validation F1: 10.722%
			Validation Recall: 25.989%
		ROC AUC validation: 0.4692212274037424
	Waktu komputasi training: 57.41352987289429 detik
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       429
           1       0.00      0.00      0.00       422
           2       0.00      0.00      0.00       365
           3       0.26      1.00      0.41       427

    accuracy                           0.26      1643
   macro avg       0.06      0.25      0.10      1643
weighted avg       0.07      0.26      0.11      1643

	Finished Training: apple_leaf_disease_1_mobilenetv3large_Adam_default_outchan1280_16batch


		Test loss: 0.154
			Test Accuracy: 11.111%
			Test Precision: 1.235

### Apple Leaf Disease

#### Default

In [7]:
# name_file = f"kmu_fed_kfold{n_splits}.{fi}.{epochs}_mobilenetv2_v0.3"
name_model = "mobilenetv3large"
model = models.MobileNetV3(inverted_residual_setting=inverted_residual_setting,
                    last_channel=last_channel,
                    num_classes=4) #MobileNetV2_CBAM_hilmi(10)
str_to_num=False
save_path = f"hasil/{name_model}"

# path_dir = "../resources"
name_data = "apple_leaf_disease"
# path = path_dir+"/isdds_skeleton/90c_v2"
# dataset_list = [path_dir+"/isdds_irhv2_nonpersonsplitv2_holdout/latih_isdds90c_irhv2.csv",
#                 path_dir+"/isdds_irhv2_nonpersonsplitv2_holdout/valid_isdds90c_irhv2.csv",
#                 path_dir+"/isdds_irhv2_nonpersonsplitv2_holdout/uji_isdds90c_irhv2.csv",]
dataset_list2 = [
    train_dataset,
    valid_dataset,
    test_dataset
]

criterion = nn.CrossEntropyLoss()

optim_params = {}
optimizer = optim.Adam
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

patience = 15#30
# counter = 0

# n_splits=5
epochs=20
batch_size= 16#8#64#32

versi = "Adam_default_outchan1280_16batch" #X, coba lr0.001

In [8]:
# workflow.run_workflow(
#     versi=versi, model=model, str_to_num=str_to_num,
#     # init_model=init_mobilenetv2(),
#     # dataset_const=dataset,
#     batch_size=batch_size,
#     # n_splits=n_splits,
#     epochs=epochs,
#     patience=patience,
#     # class_names=class_names,
#     path='',
#     name_model=name_model,
#     name_data=name_data,
#     save_path0=save_path,
#     criterion_const=criterion,
#     optim_const=optimizer, optim_params=optim_params,
#     transform_train=transform_train, transform_test=transform_test,
#     dataset_list=None, #dataset_list
#     dataset_list2=dataset_list2
# )

workflow.run_demo(
    versi=versi, model1=model, str_to_num=str_to_num,
    # init_model=init_mobilenetv2(),
    # dataset_const=dataset,
    batch_size=batch_size,
    # n_splits=n_splits,
    epochs=epochs,
    patience=patience,
    # class_names=class_names,
    path='',
    name_model=name_model,
    name_data=name_data,
    save_path0=save_path,
    criterion_const=criterion,
    optim_const=optimizer, optim_params=optim_params,
    transform_train=transform_train, transform_test=transform_test,
    dataset_list=None, #dataset_list
    dataset_list2=dataset_list2,
    # pilihan='y',
)


	Epochs 20
		Best -> epoch: 7
		Training Loss: 0.769
			Training Accuracy: 91.183%
			Training Precision: 91.174%
			Training F1: 91.177%
			Training Recall: 91.183%
		Validation loss: 7.105
			Validation Accuracy: 27.024%
			Validation Precision: 28.336%
			Validation F1: 12.890%
			Validation Recall: 27.024%
		ROC AUC validation: 0.6311718460484029
	Waktu komputasi training: 1074.1564042568207 detik
              precision    recall  f1-score   support

           0       0.60      0.01      0.01       429
           1       0.00      0.00      0.00       422
           2       0.25      0.04      0.07       365
           3       0.27      1.00      0.43       427

    accuracy                           0.27      1643
   macro avg       0.28      0.26      0.13      1643
weighted avg       0.28      0.27      0.13      1643

	Finished Training: apple_leaf_disease_20_mobilenetv3large_Adam_default_outchan1280_16batch


		Test loss: 0.165
			Test Accuracy: 11.111%
			Test Precision: 1.

#### optuna berdasarkan akurasi

In [9]:
name_model = "mobilenetv3large"
model = models.MobileNetV3(inverted_residual_setting=inverted_residual_setting,
                    last_channel=last_channel,
                    num_classes=4) #MobileNetV2_CBAM_hilmi(10)
str_to_num=False
save_path = f"hasil/{name_model}"

# path_dir = "../resources"
name_data = "apple_leaf_disease"
# path = path_dir+"/isdds_skeleton/90c_v2"
# dataset_list = [path_dir+"/isdds_irhv2_nonpersonsplitv2_holdout/latih_isdds90c_irhv2.csv",
#                 path_dir+"/isdds_irhv2_nonpersonsplitv2_holdout/valid_isdds90c_irhv2.csv",
#                 path_dir+"/isdds_irhv2_nonpersonsplitv2_holdout/uji_isdds90c_irhv2.csv",]
dataset_list2 = [
    train_dataset,
    valid_dataset,
    test_dataset
]

# criterion = nn.CrossEntropyLoss()

# optim_params = {}
# optimizer = optim.Adam
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

# patience = 15#30
# counter = 0

# n_splits=5
# epochs=20
# batch_size= 16#8#64#32

# versi = "Adam_default_outchan1280_16batch" #X, coba lr0.001

In [ ]:
optim.RMSprop(model.parameters(), )

In [18]:
# import time
# class_names = {
#   "AN": 0,
#   "DI": 1,
#   "FE": 2,
#   "HA": 3,
#   "SA": 4,
#   "SU": 5
# }

def objective_momen(trial):
    torch.cuda.empty_cache()
    
    # Generate the model.
    model = models.MobileNetV3(inverted_residual_setting=inverted_residual_setting,
                    last_channel=last_channel,
                    num_classes=4).to(device) #MobileNetV2_CBAM_hilmi(10)
    # model = MobileNetV2(num_classes=10).to(device) #phase3
    # model = MobileNetV2_CBAM_hilmi_optuna(trial).to(device)

    # Generate the optimizers.
    # epochs = trial.suggest_int("epochs", 15, 30, log=True) # phase 2
    epochs = 20
    # batch = trial.suggest_int("batch_size", 16, 64, step=16)
    batch = trial.suggest_int("batch_size", 8, 64, log=True) # phase 2
    
    # lr = trial.suggest_float("lr", 0.00001, 0.01, log=True)  # phase2
    lr = trial.suggest_float("lr", 0.0001, 0.0100, log=True)  # phase3

    wdecay = trial.suggest_float("weight_decay", 0.0000000000001, 0.0100, log=True) #phase2
    momen = trial.suggest_float("momentum", 0.8, 0.99, log=True) #phase 2
    # optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "AdamW", "RMSprop", "SGD"]) # phase2
    optimizer_name = trial.suggest_categorical("optimizer", ["SGD", "Adam"]) #
    if (optimizer_name == "Adam") or (optimizer_name == "AdamW"):
        optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr, weight_decay=wdecay)
    else:
        optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr, weight_decay=wdecay, momentum=momen)
    # optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr, weight_decay=wdecay)
    
    criterion = nn.CrossEntropyLoss()
    # alternative version
    # optimizer = trial.suggest_categorical("optimizer", [optim.Adam, optim.RMSprop, optim.SGD])

    # Get the dataset.
    # valid_dataset = pd.read_csv("kmufed_nonperson_holdout_valid.csv")
    # train_dataset = pd.read_csv("kmufed_nonperson_holdout_latih.csv")
    # valid_data = DfToDataset(valid_dataset, path, transform_test)
    # train_data = DfToDataset(train_dataset, path, transform_test)
    # train_loader = DataLoader(train_data, batch_size=batch, shuffle=False)
    # valid_loader = DataLoader(valid_data, batch_size=batch, shuffle=False)

    # test_dataset = pd.read_csv(dataset_list[1])
    # test_data = DfToDataset(test_dataset, path, transform_test)
    test_loader = DataLoader(valid_dataset, batch_size=batch, shuffle=False, num_workers=1)

    ### kfold 5.1 non person ###
    # dataset_const = pd.read_csv("kmufed_latih.csv")
    # n_splits=5
    # fi=1
    # name_file_index = f"kmu_fed_kfold{n_splits}.{fi}"

    # kfold_dir = "kfold_data_nonperson"
    # train_index = pd.read_csv(f"{kfold_dir}/latih_{name_file_index}.csv")['idx_train'].values
    # test_index = pd.read_csv(f"{kfold_dir}/valid_{name_file_index}.csv")['idx_valid'].values

    # train_groups = pd.read_csv(f"{kfold_dir}/latih_{name_file_index}.csv")['groups_train'].values#groups[train_index]
    # test_groups = pd.read_csv(f"{kfold_dir}/valid_{name_file_index}.csv")['groups_valid'].values#groups[test_index]

    # train_dataset, val_dataset = dataset_const.iloc[ train_index, :], dataset_const.iloc[ test_index, :]
    # path = "kmufed"
    # train_dataset = pd.read_csv(dataset_list[0])
    # train_data = DfToDataset(train_dataset, path, transform_train)
    # valid_data = DfToDataset(val_dataset, path, transform_test)

    # print( next(iter(train_data)))
    # plt.imshow( next(iter(train_data))[0].view(224, 224, 3) )
    # break

    # batch_size = 8  # Sesuaikan dengan kebutuhan Anda
    train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True, num_workers=9)
    # valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)
    ### ### ### ### ### ### ###
    # best_acc = 0.0
    # best_epoch = 0
    start = time.time()
    # Training of the model.
    for epoch in range(epochs):
        # start = time.time()
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            if str_to_num:
                target = torch.from_numpy( np.array([target_names[label] for label in target]) )
            # Limiting training data for faster epochs.
            # if batch_idx * batch_size >= n_train_examples:
            #     break

            data, target = data.to(device), target.to(device)

            output = model(data)
            loss = criterion(output, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(test_loader):
                if str_to_num:
                    target = torch.from_numpy( np.array([target_names[label] for label in target]) )
                # Limiting validation data.
                # if batch_idx * batch_size >= n_valid_examples:
                #     break
                data, target = data.to(device), target.to(device)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / len(test_loader.dataset)

        # report back to Optuna how far it is (epoch-wise) into the trial and how well it is doing (accuracy)
        trial.report(accuracy, epoch)  #phase 3, 4

        # # then, Optuna can decide if the trial should be pruned
        # # Handle pruning based on the intermediate value.
        if trial.should_prune(): #phase 3,4 
            raise optuna.exceptions.TrialPruned()
        
        # if round(accuracy, 2) >= round(best_acc, 2):
        #     best_acc = accuracy
        #     best_epoch = epoch
        
    end = time.time() - start
    # test_dataset = pd.read_csv("kmufed_uji.csv")
    # test_data = DfToDataset(test_dataset, path, transform_test)
    # test_loader = DataLoader(test_data, batch_size=batch, shuffle=False)

    # test_correct = 0
    # model.eval()
    # with torch.no_grad():
    #     for batch_idx, (data, target) in enumerate(test_loader):
    #         target = torch.from_numpy( np.array([class_names[label] for label in target]) )
    #         # Limiting validation data.
    #         # if batch_idx * batch_size >= n_valid_examples:
    #         #     break
    #         data, target = data.to(device), target.to(device)
    #         output = model(data)
    #         # Get the index of the max log-probability.
    #         pred = output.argmax(dim=1, keepdim=True)
    #         test_correct += pred.eq(target.view_as(pred)).sum().item()

    test_accuracy = accuracy#test_correct / len(test_loader.dataset)

    trial.report(test_accuracy, epoch)  

    # then, Optuna can decide if the trial should be pruned
    # Handle pruning based on the intermediate value.
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

    del model
    torch.cuda.empty_cache()

    # print(f"\tbest_epoch: {best_epoch}")
    print(f"\tdurasi (detik): {end}")
    # return best_acc
    return test_accuracy

In [19]:
# now we can run the experiment
name_file = f"{name_data}.{epochs}_{name_model}"#.{versi}"
n_trials=75

sampler = optuna.samplers.TPESampler()
study = optuna.create_study(study_name=f"optuna{n_trials}_{name_file}"
                            , direction="maximize", 
                            sampler=sampler,
                            # storage=f'sqlite:///../hasil/{name_model}/optuna{n_trials}_{name_file}.db')
                            storage=f'sqlite:///hasil/{name_model}/optuna{n_trials}_{name_file}.db',
                            load_if_exists=True)
study.optimize(objective_momen, n_trials=n_trials)

pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2025-11-15 20:41:55,214] Using an existing study with name 'optuna75_apple_leaf_disease.20_mobilenetv3large' instead of creating a new one.
/home/andre/.conda/envs/smt11/lib/python3.12/site-packages/optuna/trial/_trial.py:503: UserWarning: The reported value is ignored because this `step` 19 is already reported.
  warnings.warn(
[I 2025-11-15 20:55:38,964] Trial 7 finished with value: 0.996956786366403 and parameters: {'batch_size': 9, 'lr': 0.005782939669342303, 'weight_decay': 0.00015087257328730072, 'momentum': 0.801320807259348, 'optimizer': 'SGD'}. Best is trial 7 with value: 0.996956786366403.


	durasi (detik): 823.6117918491364


[I 2025-11-15 21:08:54,141] Trial 8 finished with value: 0.9050517346317711 and parameters: {'batch_size': 12, 'lr': 0.0003505225461374033, 'weight_decay': 2.8146119039002143e-08, 'momentum': 0.805300052776449, 'optimizer': 'SGD'}. Best is trial 7 with value: 0.996956786366403.


	durasi (detik): 795.0676860809326


[I 2025-11-15 21:22:36,649] Trial 9 finished with value: 0.9939135727328059 and parameters: {'batch_size': 10, 'lr': 0.006531294991590264, 'weight_decay': 4.5810609823322085e-05, 'momentum': 0.8243717564953416, 'optimizer': 'SGD'}. Best is trial 7 with value: 0.996956786366403.


	durasi (detik): 822.396892786026


[W 2025-11-15 21:34:32,233] Trial 10 failed with parameters: {'batch_size': 12, 'lr': 0.0034163737902020487, 'weight_decay': 1.635745847821771e-11, 'momentum': 0.8360130580745505, 'optimizer': 'SGD'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/andre/.conda/envs/smt11/lib/python3.12/site-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_131845/2784721663.py", line 97, in objective_momen
    data, target = data.to(device), target.to(device)
                   ^^^^^^^^^^^^^^^
KeyboardInterrupt
[W 2025-11-15 21:34:32,234] Trial 10 failed with value None.


KeyboardInterrupt: 

In [20]:
# joblib.dump(study, f"{savepath}xgb_optuna_study_batch.pkl")   # save study

# # to load it:
# jl = joblib.load(f"{savepath}xgb_optuna_study_batch.pkl")

# name_file = f"{name_data}.{epochs}_{name_model}.{versi}"

joblib.dump(study, f"hasil/{name_model}/optuna{n_trials}_{name_file}.pkl")

['hasil/mobilenetv3large/optuna75_apple_leaf_disease.20_mobilenetv3large.pkl']

In [ ]:
name_file = f"{name_data}.{epochs}_{name_model}"#.{versi}"
n_trials=125

sampler = optuna.samplers.TPESampler()
study = optuna.create_study(study_name=f"optuna{n_trials}_{name_file}"
                            , direction="maximize", 
                            sampler=sampler,
                            # storage=f'sqlite:///../hasil/{name_model}/optuna{n_trials}_{name_file}.db')
                            storage=f'sqlite:///hasil/{name_model}/optuna{n_trials}_{name_file}.db',
                            load_if_exists=True)


[I 2025-11-15 17:58:27,286] Using an existing study with name 'optuna125_apple_leaf_disease.20_mobilenetv3small' instead of creating a new one.


In [21]:
optuna.visualization.plot_param_importances(study)

In [22]:
optuna.visualization.plot_intermediate_values(study)

#### Menggunakan hasil optuna berdasarkan akurasi

In [5]:
# name_model = "mobilenetv2"
# model = MobileNetV2(num_classes=10)
# str_to_num=False
# save_path = f"../hasil/{name_model}"

# path_dir = "../resources"
# name_data = "isdds90c_irhv2_holdout_nonpersonsplitv2"
# path = path_dir+"/isdds_skeleton/90c_v2"
# dataset_list = [path_dir+"/isdds_irhv2_nonpersonsplitv2_holdout/latih_isdds90c_irhv2.csv",
#                 path_dir+"/isdds_irhv2_nonpersonsplitv2_holdout/valid_isdds90c_irhv2.csv",
#                 path_dir+"/isdds_irhv2_nonpersonsplitv2_holdout/uji_isdds90c_irhv2.csv",]


# criterion = nn.CrossEntropyLoss()

# # Best trial:
# #   Value:  0.8391472868217055
# #   Params: 
# #     batch_size: 12
# #     lr: 0.0002464611134002072
# #     weight_decay: 0.003174735064998515
# #     momentum: 0.9099636145589376
# #     optimizer: Adam


# optim_params = {"lr":0.0002464611134002072, "weight_decay":0.003174735064998515}
# optimizer = optim.Adam
# # scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

# patience = 15
# # counter = 0

# # n_splits=5
# epochs=75 #50
# batch_size= 12

# versi = "Adam_lr0.0002464611134002072_wdecay0.003174735064998515_outchan1280_12batch" #X, coba lr0.001

# name_file = f"kmu_fed_kfold{n_splits}.{fi}.{epochs}_mobilenetv2_v0.3"
name_model = "mobilenetv3large"
model = models.MobileNetV3(inverted_residual_setting=inverted_residual_setting,
                    last_channel=last_channel,
                    num_classes=4) #MobileNetV2_CBAM_hilmi(10)
str_to_num=False
save_path = f"hasil/{name_model}"

# path_dir = "../resources"
name_data = "apple_leaf_disease"
# path = path_dir+"/isdds_skeleton/90c_v2"
# dataset_list = [path_dir+"/isdds_irhv2_nonpersonsplitv2_holdout/latih_isdds90c_irhv2.csv",
#                 path_dir+"/isdds_irhv2_nonpersonsplitv2_holdout/valid_isdds90c_irhv2.csv",
#                 path_dir+"/isdds_irhv2_nonpersonsplitv2_holdout/uji_isdds90c_irhv2.csv",]
dataset_list2 = [
    train_dataset,
    valid_dataset,
    test_dataset
]

# {'batch_size': 9, 'lr': 0.005782939669342303, 'weight_decay': 0.00015087257328730072, 'momentum': 0.801320807259348, 'optimizer': 'SGD'}


criterion = nn.CrossEntropyLoss()

optim_params = {"lr": 0.005782939669342303, "weight_decay": 0.00015087257328730072, "momentum": 0.801320807259348}
optimizer = optim.SGD
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

patience = 15#30
# counter = 0

# n_splits=5
epochs=20 #100
batch_size= 9

versi = "SGD_lr0.005782939669342303_wdecay0.00015087257328730072_momen0.801320807259348_outchan1280_9batch"

In [6]:
workflow.run_demo(
    versi=versi, model1=model, str_to_num=str_to_num,
    # init_model=init_mobilenetv2(),
    # dataset_const=dataset,
    batch_size=batch_size,
    # n_splits=n_splits,
    epochs=epochs,
    patience=patience,
    # class_names=class_names,
    path='',
    name_model=name_model,
    name_data=name_data,
    save_path0=save_path,
    criterion_const=criterion,
    optim_const=optimizer, optim_params=optim_params,
    transform_train=transform_train, transform_test=transform_test,
    dataset_list=None, #dataset_list
    dataset_list2=dataset_list2,
    # pilihan='y',
)


	Epochs 20
		Best -> epoch: 20
		Training Loss: 0.733
			Training Accuracy: 76.193%
			Training Precision: 76.304%
			Training F1: 76.189%
			Training Recall: 76.193%
		Validation loss: 7.316
			Validation Accuracy: 25.989%
			Validation Precision: 6.754%
			Validation F1: 10.722%
			Validation Recall: 25.989%
		ROC AUC validation: 0.44946872545812344
	Waktu komputasi training: 1194.2261288166046 detik
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       429
           1       0.00      0.00      0.00       422
           2       0.00      0.00      0.00       365
           3       0.26      1.00      0.41       427

    accuracy                           0.26      1643
   macro avg       0.06      0.25      0.10      1643
weighted avg       0.07      0.26      0.11      1643

	Finished Training: apple_leaf_disease_20_mobilenetv3large_SGD_lr0.005782939669342303_wdecay0.00015087257328730072_momen0.801320807259348_outchan1280_9batch




In [11]:
model

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [22]:
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import os


# os.environ['MASTER_ADDR'] = 'localhost'
# os.environ['MASTER_PORT'] = '12355'
# acc = torch.accelerator.current_accelerator()
# backend = torch.distributed.get_default_backend_for_device(acc)
# # initialize the process group
# dist.init_process_group(backend, rank=0, world_size=1)


model = DDP(model.to('cuda'), device_ids=[0])

In [25]:
model.module.features[-1]

Conv2dNormActivation(
  (0): Conv2d(96, 576, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(576, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  (2): Hardswish()
)

#### optuna berdasarkan loss